In [88]:
import pandas as pd
from litellm import completion
import litellm
import os
litellm.enable_json_schema_validation = True
litellm.set_verbose = True
from judge import Judge
import json
from litellm import ModelResponse
from dotenv import load_dotenv
load_dotenv()
import time

In [89]:
context_rows = 40

In [91]:
try:
    df = pd.read_csv("gemini with context.csv")
except FileNotFoundError:
    df = pd.read_csv("base accuracy 100 gemini.csv")

In [92]:
from pydantic import BaseModel
class Answer(BaseModel):
  reasoning: str
  answer: str

response_schema_answer = {
    "type": "object",
    "properties": {
        "reasoning": {
            "type": "string"
        },
        "answer": {
            "type": "string"
        }
    },
    "required": ["reasoning", "answer"]
}



In [93]:
def generate_training_messages(df, start_idx:int, num_rows:int):
    end_idx = min(start_idx + num_rows, len(df))
    messages = [{
                'role': 'system',
                'content': '''Be a helpful assistant.
                You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                and your reasoning in the reasoning key. 
                
                example:
                user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                assistant output: 
                {
                    "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                    "answer" : "area is 12 cm squared."
                }
                ''',
            }]
    for idx in range(start_idx, end_idx):
        row = df.iloc[idx]
        
        # Base messages that are common for all examples
        messages.extend([
            {
                'role': 'user',
                'content': row['problem']
            },
            {
                'role': 'assistant',
                'content': row['llm_raw_response']
            }
        ])
        
        # Add feedback based on correctness
        if not row['is_correct']:
            messages.append({
                'role': 'user',
                'content': f"Let me correct this. The right answer is {row['answer']}. Let's understand the solution: {row['solution']}"
            })
        else:
            messages.append({
                'role': 'user',
                'content': "Good job! Your reasoning and answer are correct!"
            })
        messages.append({
            'role': 'assistant',
            'content': "Understood. I will keep this in mind"
        })
            
       
    
    return messages

In [94]:
training_data = generate_training_messages(df,start_idx=0,num_rows=context_rows)

In [ ]:
len(training_data)

In [96]:
def test_process_math_problems_te(df, start_idx:int, end_idx:int):
    idx = start_idx
    while idx<end_idx:
        try:
            problem = df.iloc[idx]['problem']
            answer = df.iloc[idx]["answer"]
            messages = training_data + [
                {
                    'role': 'system',
                    'content': '''Be a helpful assistant.
                    You need to just give me the final answer and no other text. Don't tell the steps. Just give the final output for the answer key 
                    and your reasoning in the reasoning key. 
                    
                    example:
                    user query: What is the area of a rectangle with length 3cm and breadth 4cm. 
                    assistant output: 
                    {
                        "reasoning": "area of a rectangle is length * breadth, so here it will be 3cm*4cm which is 12cm squared."
                        "answer" : "area is 12 cm squared."
                    }
                    ''',
                },
                {
                    'role': 'user',
                    'content': problem,
                }]
            
            response = completion(model='gemini/gemini-1.5-flash', messages=messages,
                response_format={"type": "json_object", "response_schema": response_schema_answer}
            )
            answer_content = response.choices[0]['message']['content']

            answer_obj = Answer.model_validate_json(answer_content)
            llm_answer = answer_obj.answer

            judge = Judge(model='gemini/gemini-2.0-flash')
            answer_correctness_obj = judge.prediction(query=df.iloc[idx]['problem'],answer1=answer,answer2=llm_answer)
            
            df.at[idx, f'llm_raw_response_test_context_{context_rows}'] = answer_content
            df.at[idx, f'llm_answer_test_{context_rows}'] = llm_answer
            df.at[idx, f'is_correct_test_{context_rows}'] = answer_correctness_obj.correct
            
            print(f"Processed row {idx}")
            idx = idx + 1
        except Exception as e:
            time.sleep(60)
        


In [98]:
test_process_math_problems_te(df,start_idx=80,end_idx=100)

22:27:31 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_mime_type': 'application/json', 'response_schema': {'type': 'object', 'properties': {'reasoning': {'type': 'string'}, 'answer': {'type': 'string'}}, 'required': ['reasoning', 'answer']}}


22:27:49 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "{\n\"answer\": \"997\",\n\"reasoning\": \"We are given the equation log(kx) = 2log(x+2). Using the properties of logarithms, we can rewrite this as log(kx) = log((x+2)^2). Since the logarithm is a one-to-one function, we can equate the arguments: kx = (x+2)^2. This simplifies to kx = x^2 + 4x + 4. Rearranging the terms, we get x^2 + (4-k)x + 4 = 0. This is a quadratic equation in x. For the quadratic equation to have exactly one real solution, its discriminant must be zero. The discriminant is (4-k)^2 - 4(1)(4) = k^2 - 8k + 16 - 16 = k^2 - 8k = k(k-8). Setting the discriminant to zero, we have k(k-8) = 0, which gives k = 0 or k = 8. However, if k = 0, then the original equation becomes log(0) which is undefined. Therefore, we must have k = 8. If k = 8, the equation becomes x^2 - 4x + 4 = 0, which factors as (x-2)^2 = 0, so x = 2. The given interval for k is [-500, 500]. The va

22:27:51 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


...

RAW RESPONSE:
{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "{\n  \"answer\": \"2k + 2\",\n  \"reasoning\": \"To simplify the expression (-k + 4) + (-2 + 3k), we combine like terms. The terms with 'k' are -k and 3k, and the constant terms are 4 and -2. Combining the 'k' terms, we get -k + 3k = 2k. Combining the constant terms, we get 4 + (-2) = 2. Therefore, the simplified expression is 2k + 2.\"\n}"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.026433908939361574
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 25917,
    "candidatesTokenCount": 120,
    "totalTokenCount": 26037,
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 25917
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 120
      }
    ]
  },
  "modelVersion": "gemini-1.5-flash"
}



SYNC kwargs[caching]: False

22:30:36 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "{\n  \"correct\": true\n}"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.061208652125464544
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 246,
    "candidatesTokenCount": 9,
    "totalTokenCount": 255,
    "promptTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 246
      }
    ],
    "candidatesTokensDetails": [
      {
        "modality": "TEXT",
        "tokenCount": 9
      }
    ]
  },
  "modelVersion": "gemini-2.0-flash"
}



Processed row 98
SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
Final returned optional params: {'response_mime_type': 'application/json', 'response_schema': {'type': 'object', 'properties': {'reasoning': {'type': 'string'}, 'answer': {'type': 'string'}}, 'required': ['reasoning', 'answer']}}


22:30:44 - LiteLLM:WARNING: utils.py:430 - `litellm.set_verbose` is deprecated. Please set `os.environ['LITELLM_LOG'] = 'DEBUG'` for debug logs.


RAW RESPONSE:
{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "{\n  \"answer\": \"[0,1,1]\",\n  \"reasoning\": \"Let \\\\mathbf{v} = \\\\begin{pmatrix} x \\\\ y \\\\ z \\\\end{pmatrix}.  Then \\\\mathbf{a} \\\\cdot \\\\mathbf{v} = x + y + z = 2, and \\\\mathbf{a} \\\\times \\\\mathbf{v} = \\\\begin{pmatrix} y - z \\\\ z - x \\\\ x - y \\\\end{pmatrix} = \\\\begin{pmatrix} 1 \\\\ -2 \\\\ 1 \\\\end{pmatrix}.  Then y - z = 1, z - x = -2, and x - y = 1.  From x - y = 1, x = y + 1.  From z - x = -2, z = x - 2 = y - 1.  From y - z = 1, y - (y - 1) = 1, which is true.  Then x + y + z = (y + 1) + y + (y - 1) = 3y = 2, so y = 2/3.  Then x = 5/3 and z = -1/3.  Then \\\\mathbf{v} = \\\\begin{pmatrix} 5/3 \\\\ 2/3 \\\\ -1/3 \\\\end{pmatrix}.  However, this vector doesn't work.  Let \\\\mathbf{v} = \\\\begin{pmatrix} x \\\\ y \\\\ z \\\\end{pmatrix}.  Then x + y + z = 2, and\\n\\\\begin{pmatrix} y - z \\\\ z - x \\\\ x - y \\\\end{pmatrix} = \\\\begin

In [99]:
df.to_csv('gemini with context.csv')

In [100]:
df = pd.read_csv('gemini with context.csv')

In [101]:
df_test = df.iloc[context_rows:100]

In [102]:
df_test['is_correct'].value_counts()

is_correct
False    37
True     23
Name: count, dtype: int64

In [103]:
df_test[f'is_correct_test_{context_rows}'].value_counts()

is_correct_test_40
False    34
True     26
Name: count, dtype: int64